In [1]:
# Install Python 3.10.0: https://www.python.org/ftp/python/3.10.0/python-3.10.0-amd64.exe
# (optionally) upgrade pip: python3 -m pip install --upgrade pip

# pip install wheel
# pip install unstructured[local-inference]
# pip install poppler-utils
# pip install git+https://github.com/facebookresearch/detectron2.git

# Download tesseract-ocr: https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-w64-setup-5.3.1.20230401.exe
# Download poppler: https://github.com/oschwartz10612/poppler-windows/releases/download/v23.01.0-0/Release-23.01.0-0.zip
# Put poppler in path: C:\Program Files\Poppler\Library\bin

# Pinecone optional
# pip install pinecone-client[grpc] chromadb openai langchain tiktoken python-dotenv


In [1]:
pip install wheel


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached wheel-0.40.0-py3-none-any.whl (64 kB)


In [2]:
pip install unstructured[local-inference]


     ---------------------------------------- 1.3/1.3 MB 12.1 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 23.0 MB/s eta 0:00:00
     -------------------------------------- 101.8/101.8 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 16.0 MB/s eta 0:00:00
     ---------------------------------------- 250.0/250.0 kB ? eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 10.1/10.1 MB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 93.9/93.9 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 19.2/19.2 MB 17.7 MB/s et


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install poppler-utils

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install git+https://github.com/facebookresearch/detectron2.git

  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\bigyo\appdata\local\temp\pip-req-build-8ilbdw3t
  Resolved https://github.com/facebookresearch/detectron2.git to commit c39b2d04891495aeb054c7f932ec906211979586
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
     ---------------------------------------- 5.6/5.6 MB 20.0 MB/s eta 0:00:00
  Using cached fvcore-0.1.5.post20221221.tar.gz (50 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached iopath-0.1.9-py3-none-any.whl (27 kB)
     -------------------------------------- 154.5/154.5 kB 9.6 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 20.4 MB/s eta 0:00:00
  Using cached packaging-23.1-py3-none-any.whl (48 kB)
  Using cached pathspec-0.11.1-py3-none-any.whl (29 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB

  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git 'C:\Users\Bigyo\AppData\Local\Temp\pip-req-build-8ilbdw3t'

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
Put poppler in path: C:\Program Files\Poppler\Library\bin

In [5]:
pip install pinecone-client[grpc] chromadb openai langchain tiktoken python-dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.12.3 requires protobuf>=3.19.6, but you have protobuf 3.19.3 which is incompatible.

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached pinecone_client-2.2.1-py3-none-any.whl (177 kB)
     ---------------------------------------- 46.4/46.4 kB 2.3 MB/s eta 0:00:00
     ------------------------------------- 579.4/579.4 kB 12.1 MB/s eta 0:00:00
  Using cached loguru-0.7.0-py3-none-any.whl (59 kB)
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)
  Using cached googleapis_common_protos-1.59.0-py2.py3-none-any.whl (223 kB)
     ---------------------------------------- 99.8/99.8 kB ? eta 0:00:00
     -------------------------------------- 162.7/162.7 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 223.4/223.4 kB ? eta 0:00:00
     --------------------------

In [4]:
import pytesseract as pt

pt.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader


### Load your data


In [5]:
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")


In [6]:
# data = loader.load()


In [1]:
import pickle

# with open("../data/field-guide-to-data-science.pkl", "wb") as f:
#     pickle.dump(data, f)

with open("../data/lecture_notes.pkl", "rb") as f:
    data = pickle.load(f)


In [2]:
print(f"You have {len(data)} document(s) in your data")
print(f"There are {len(data[0].page_content)} characters in your document")


You have 1 document(s) in your data
There are 481103 characters in your document


### Chunk your data up into smaller documents


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)


In [4]:
print(f"Now you have {len(texts)} documents")


Now you have 618 documents


### Create embeddings of your documents to get ready for semantic search


In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
import chromadb


In [6]:
from dotenv import load_dotenv

load_dotenv()

import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [7]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [8]:
collection_name = "lecture_notes"
docsearch = Chroma.from_texts(
    [t.page_content for t in texts], embeddings, collection_name=collection_name
)


Using embedded DuckDB without persistence: data will be transient


In [9]:
query = "What is this book about?"
docs = docsearch.similarity_search(query, include_metadata=True)


In [10]:
# Here's an example of the first document that was returned
docs[0].page_content[:250]


'Each chapter of this document provides a literature section that describes suggested reading. The suggested reading is not part of the exam. It is for students who are interested in background information and/or a different perspective on or presenta'

### Query those docs to get your answer back


In [11]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain


In [12]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


In [25]:
query = """An online music store allows its members to listen to music with embedded ads for free and to download music without ads for a fee. Members can also recommend songs to other members and get a free ringtone if at least five people listen to a song based on this recommendation. Do the first steps in a basic 'security requirements engineering' for this scenario: identify actors, their interests and interdependencies. Also find attackers and their goals. (When we discuss security tools in chapters 8, 9, 10, we will come back to this exercise multiple times. In separate exercises you will be asked to enhance the requirements you extract here and the security design step-by-step by using the new knowledge you gain, e.g., with potential countermeasures, threats, techniques you learn in those chapters.)
"""

docs = docsearch.similarity_search(query, include_metadata=True)
chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)


' The security issue in this article is related to protecting the online music store from attackers and their goals. This includes protecting the store from unauthorized access, data breaches, and other malicious activities. The security requirements engineering process should identify the actors, their interests, and interdependencies, as well as potential attackers and their goals. The security design should then be enhanced by considering potential threats and techniques introduced in chapters 8, 9, and 10, and by applying countermeasures, threats, and techniques to protect the store from attackers.'

In [18]:
# from langchain.chains.summarize import load_summarize_chain

# chain = load_summarize_chain(llm, chain_type="map_reduce")
# chain.run(docs)
